In [12]:
from paragraph_feature import *
from preprocess import *
from sentence_feature import *
from word_feature import *
import pandas as pd
import numpy as np
import re

In [11]:
industry = pd.read_csv("C:/Users/12968/Desktop/数据科学实战-stock prediction/industry.csv", encoding="gbk")
sample = list(industry['content'])[0]
sample

'    石油石化行业中，4月23日当周北上资金个股净流入主力为恒力石化、东方盛虹、荣盛石化、桐昆股份、华锦股份，其中，恒力石化净流入1.92亿元；当周北上资金个股净流出前三为中国石化、中国石油、恒逸石化。 ???\r\r\r\n    基础化工行业中，4月23日当周北上资金个股净流入主力为国瓷材料、天赐材料、恩捷股份、玲珑轮胎、华鲁恒升，其中，国瓷材料净流入4.53亿元；北上资金个股净流出前三为万华化学、龙蟒佰利、光威复材。 ???\r\r\r\n    新材料行业中，4月23日当周北上资金个股净流入主力为南山铝业、中国宝安、中泰化学、云海金属、方大炭素，其中，南山铝业净流入1.4亿元；北上资金个股净流出前三为光威复材、金发科技、彤程新材。 ???\r\r\r\n    4月23日当周，石油石化行业北上资金测算累计净流出173百万，其中配置型、交易型和托管于内港资机构的资金净流入124、-182、-115百万元；基础化工行业北上资金测算累计净流入1310百万，其中配置型、交易型和托管于内港资机构的资金净流入2763、-1306、-147百万元；新材料行业北上资金测算累计净流入575百万，其中配置型、交易型和托管于内港资机构的资金净流入298、369、-93百万元。 ???\r\r\r\n    截至2021年4月23日，北上资金持有A股石油石化行业市值229亿，其中配置型、交易型和托管于内港资机构的资金分别持有183、36、10亿元；北上资金持有A股基础化工行业市值897亿，其中配置型、交易型和托管于内港资机构的资金分别持有695、166、37亿元；北上资金持有A股新材料行业市值178亿，其中配置型、交易型和托管于内港资机构的资金分别持有117、44、18亿元。'

In [31]:
import jieba
from nltk.tree import Tree
import re
import numpy as np
import logging
from stanfordcorenlp import StanfordCoreNLP
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from summarize import *


PATH = '../词典/'


class ParseTree:

    def __init__(self, text):
        self.text = text  # 传入的文本
        # 定义模型
        self.nlp = StanfordCoreNLP(r'E:/py/stanford-corenlp-4.2.0', lang='zh', quiet=False, logging_level=logging.DEBUG)
        # 分句
        self.sentences = self.preprocess()
        self.stopwords = []

    def load_dicts(self):
        stop = PATH + 'stop1205.txt'
        self.stopwords = self.dict_load(stop)

#     def preprocess(self):
#         """
#         预处理：
#         1. 去除换行符、多余的空格、百分号
#         2. 分句，存入列表
#         :return:返回句子列表
#         """
#         sentences = []
#         self.text = re.sub('%', '', re.sub(' ', '', re.sub('\xa0\xa0\xa0\r\n', '', self.text)))
#         start = 0
#         for i in range(len(self.text)):
#             if self.text[i] in ['。', '!', '；', '？', '……']:
#                 sentences.append(self.text[start:i + 1])
#                 start = i + 1
#         return sentences

    def preprocess(self):
        """
        把文本处理成摘要句子列表
        """
        return get_sum(self.text)
        
    def tree(self, sentence):
        res = self.nlp.parse(sentence)
        # nlp.close()
        return res

    def sum_of_heights(self):
        """
        计算整篇文本的每句话构成的语法分析树的高度之和
        :return: 高度之和
        """
        sumHeights = []
        for sentence in self.sentences:
            res = self.tree(sentence)  # 语法树，是个字符串
            sumHeights.append(len(res.split("\r\n")))
        return np.sum(sumHeights)

    def avg_height(self):
        """
        这篇文章的每句话的语法分析树的平均高度
        :return:
        """
        return self.sum_of_heights() / len(self.sentences)

    def no_less_than_16(self):
        """
        计算整篇文本的高度不大于16的语法分析树的个数
        :return:
        """
        num = 0
        for sentence in self.sentences:
            res = self.tree(sentence)
            if len(res) >= 16:
                num += 1
        return num

    def no_less_than_16_percent(self):
        """
        高度不低于１６的语法分析树的比例
        :return:
        """
        return self.no_less_than_16() / len(self.sentences)

    def nodes_sum(self):
        """
        总节点数
        :return:
        """
        node_sums = []
        for sentence in self.sentences:
            res = self.nlp.parse(sentence)
            result = -1  # 去除root
            for i in res:
                if i == '(':
                    result += 1
            node_sums.append(result)
        return np.sum(node_sums)

    def avg_nodes_sentence(self):
        """
        每句话的平均节点
        :return:
        """
        return self.nodes_sum() / len(self.sentences)

    def seg_sentence(self, sentence):
        """
        输入字符串，返回分词后的列表
        :param sentence:
        :return:
        """
        jieba.load_userdict('../词典/userdict.txt')
        sentence_seged = jieba.cut(sentence.strip())
        outstr = ''
        for word in sentence_seged:
            if word not in self.stopwords:
                if word != '\t':
                    outstr += word
                    outstr += " "
        return outstr.split(' ')

    def avg_nodes_word(self):
        """
        每个词的平均节点
        :return:
        """
        # 计算有几个词
        num = 0
        for sentence in self.sentences:
            sentence = self.seg_sentence(sentence)
            num += len(sentence)
        return self.nodes_sum() / num

    def np_sum(self):
        """
        计算整篇文章里的名词短语个数
        :return:
        """
        num = 0
        for sentence in self.sentences:
            res = self.tree(sentence).split("\r\n")
            for i in res:
                if 'NP' in i:
                    num += 1
        return num

    def avg_np(self):
        """
        语法分析树的平均名词短语个数
        :return:
        """
        return self.np_sum() / len(self.sentences)

    def vp_sum(self):
        """
        计算整篇文章里的动词短语个数
        :return:
        """
        num = 0
        for sentence in self.sentences:
            res = self.tree(sentence).split("\r\n")
            for i in res:
                if 'VP' in i:
                    num += 1
        return num

    def avg_vp(self):
        """
        语法分析树的平均动词短语个数
        :return:
        """
        return self.vp_sum() / len(self.sentences)

    def adjp_sum(self):
        """
        计算整篇文章里的形容词短语个数
        :return:
        """
        num = 0
        for sentence in self.sentences:
            res = self.tree(sentence).split("\r\n")
            for i in res:
                if 'ADJP' in i:
                    num += 1
        return num

    def avg_adjp(self):
        """
        语法分析树的平均形容词短语个数
        :return:
        """
        return self.adjp_sum() / len(self.sentences)

    def get_res(self):
        res = {}
        res['sum_height'] = self.sum_of_heights()
        res['height_16'] = self.no_less_than_16()
        res['sum_node'] = self.nodes_sum()
        res['sum_n'] = self.np_sum()
        res['sum_v'] = self.vp_sum()
        res['sum_adj'] = self.adjp_sum()
        res['avg_height'] = self.avg_height()
        res['16_ratio'] = self.no_less_than_16_percent()
        res['avg_node'] = self.avg_nodes_sentence()
        res['word_avg_node'] = self.avg_nodes_word()
        res['avg_n'] = self.avg_np()
        res['avg_v'] = self.avg_vp()
        res['avg_adj'] = self.avg_adjp()
        return res

In [32]:
tree = ParseTree(sample)
tree.get_res()

INFO:root:Initializing native server...
INFO:root:java -Xmx4g -cp "E:\py\stanford-corenlp-4.2.0\*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9006
INFO:root:Server shell PID: 10536
INFO:root:The server is available.
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'zh'}
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:9006
DEBUG:urllib3.connectionpool:http://127.0.0.1:9006 "POST /?properties=%7B%27annotators%27%3A+%27pos%2Cparse%27%2C+%27outputFormat%27%3A+%27json%27%7D&pipelineLanguage=zh HTTP/1.1" 200 40598
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'zh'}
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:9006
DEBUG:urllib3.connectionpool:http://127.0.0.1:9006 "POST /?properties=%7B%27annotators%27%3A+%27pos%2Cparse%27%2C+%27outputFormat%27%3A+%27json%27%7D&pipelineLanguage=zh HTTP/1.1" 200 40598
INFO:root:{'properties': 

{'sum_height': 65,
 'height_16': 1,
 'sum_node': 105,
 'sum_n': 21,
 'sum_v': 18,
 'sum_adj': 2,
 'avg_height': 65.0,
 '16_ratio': 1.0,
 'avg_node': 105.0,
 'word_avg_node': 2.019230769230769,
 'avg_n': 21.0,
 'avg_v': 18.0,
 'avg_adj': 2.0}